In [47]:
from PIL import ImageGrab, ImageOps
import pyautogui
import time
import numpy as np

In [48]:
class Coordinates():
    replayBtn = (1000, 500)
    replayBox = (915,492,1007,564)
    imediate_box = (214,552,500,689) #obstáculo logo a frente
    long_box = (214,552,1919,689) #tudo a frente
    sky_box = (2,68,7,71) #céu, checa se é dia ou noite

def restartGame():
    pyautogui.click(Coordinates.replayBtn)

def pressSpace():
    pyautogui.keyDown('space')
    time.sleep(.05)
    pyautogui.keyUp('space')

def Replay_button():
    rpl_img = ImageOps.grayscale(ImageGrab.grab(Coordinates.replayBox))
    rpl_img_value=np.sum(np.array(rpl_img))
    return rpl_img_value

def state_data():
    # Começa olhando para o que tem logo a frente:
    image_1 = ImageGrab.grab(Coordinates.imediate_box)
    grayImage_1 = ImageOps.grayscale(image_1)
    num_1 = np.sum(np.array(grayImage_1.getcolors()))
    # Agora vai ver tudo o que tem a frente duas vezes para ter uma ideia da velocidade
    image_2 = ImageGrab.grab(Coordinates.long_box)
    grayImage_2 = ImageOps.grayscale(image_2)
    num_2 = np.array(grayImage_2.getcolors())
    time.sleep(.01)
    image_3 = ImageGrab.grab(Coordinates.long_box)
    grayImage_3 = ImageOps.grayscale(image_3)
    num_3 = np.array(grayImage_3.getcolors())
    dif = np.sum(num_2)-np.sum(num_3)
    #agora vamos pegar o céu para ver se é dia ou noite
    image_4 = ImageGrab.grab(Coordinates.sky_box)
    grayImage_4 = ImageOps.grayscale(image_4)
    num_4 = np.sum(np.array(grayImage_4.getcolors()))
    return num_1, dif, num_4
    
def find_survivors(gens, names):
    payoffs = {} #resultados das espécies
    survivors = {} #dicionário dos sobreviventes
    for i in range(4):
        payoffs[names[i]] = len(gens[names[i]]) #o payoff de cada espécie é a quantidade de estados que ela descobriu por ter sobrevivido mais
    listed = [payoffs[names[i]] for i in range(4)]
    payoff_vals = sorted(listed)
    sur_pay = payoff_vals[-2:] #os payoffs vão ser os dois maiores valores
    sur_spec = [i for i, x in enumerate(listed) if x in sur_pay] #achar quais foram as melhores espécies
    return sur_spec

In [49]:
def teste(gens,state_space,names,start_time):
    payoffs = {}
    for i in range(4):
        restartGame()
        while True:
            # Começamos checando o estado atual
            current_state = (state_data()[0], state_data()[1], state_data()[2])
            # Se é um estado desconhecido vamos adicioná-lo ao espaço de estados
            if current_state not in state_space[names[i]]:
                state_space[names[i]] += [current_state]
                # Agora vamos escolher uma ação dada a espécie
                if i == 0:
                    gens[names[i]] += [0] #primeira espécie sempre escolhe 0 ao encontrar um estado novo
                if i == 1:
                    gens[names[i]] += [1] #segunda espécie sempre escolhe 1 ao encontrar um estado novo
                if i == 2:
                    gens[names[i]] += list(np.random.choice([0, 1], size=(1), p=[1/3, 2/3])) #terceira espécie escolhe 1 com probabilidade 2/3 ao encontrar um estado novo
                if i == 3:
                    gens[names[i]] += list(np.random.choice([0, 1], size=(1), p=[2/3, 1/3]))
            state_pos = state_space[names[i]].index(current_state) #ache a posição do estado atual na lista de estados conhecidos
            if gens[names[i]][state_pos] == 1: #se a política é 1, pule!
                pressSpace()
            if Replay_button() == 819900: #se apareceu o botão de replay
                break
    return gens, state_space

In [50]:
def new_generation(gens, survivors, names, state_space):
    new_gens = {}
    new_state_space = {}
    survivors = survivors[:2]
    size_1 = len(gens[names[survivors[0]]])
    size_2 = len(gens[names[survivors[1]]])
    cross_num = int(np.ceil(size_2/5)) # at lead 20% crossover
    pos_1 = np.random.choice(size_1, cross_num, replace=False) #gene positions to be swapped of the first survivor's first child
    pos_2 = np.random.choice(size_1, cross_num, replace=False) #gene positions to be swapped of the first survivor's second child
    pos_3 = np.random.choice(size_2, cross_num, replace=False) #same for the second survivor...
    pos_4 = np.random.choice(size_2, cross_num, replace=False)
    for i in range(2):
        new_gens[names[2*i]] = gens[names[survivors[i]]].copy() #we start by making the children the copy of their parents
        new_state_space[names[2*i]] = state_space[names[survivors[i]]].copy() #need to copy the state space as well as we're insterested in the relationship between state and action
        new_gens[names[2*i+1]] = gens[names[survivors[i]]].copy()
        new_state_space[names[2*i+1]] = state_space[names[survivors[i]]].copy()
    #crossover:
    for k in range(cross_num):
        new_gens[names[0]][pos_1[k]] = gens[names[survivors[1]]].copy()[pos_3[k]] #the new genes for specie 1, pos_1 is going to be swapped with the genes from survivor 2 pos_3
        new_state_space[names[0]][pos_1[k]] = state_space[names[survivors[1]]].copy()[pos_3[k]] #states must also be swapped since the policy is a relationship between state and action
        
        new_gens[names[1]][pos_2[k]] = gens[names[survivors[1]]].copy()[pos_4[k]]
        new_state_space[names[1]][pos_2[k]] = state_space[names[survivors[1]]].copy()[pos_4[k]]
        
        new_gens[names[2]][pos_3[k]] = gens[names[survivors[0]]].copy()[pos_1[k]]
        new_state_space[names[2]][pos_3[k]] = state_space[names[survivors[0]]].copy()[pos_1[k]]
        
        new_gens[names[3]][pos_4[k]] = gens[names[survivors[0]]].copy()[pos_2[k]]
        new_state_space[names[3]][pos_4[k]] = state_space[names[survivors[0]]].copy()[pos_2[k]]
    #mutations:
    mut_num = int(np.ceil(size_2/20)) #at least 5% mutation
    mut_pos_1 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False) #randomly choose positions to chance policy
    for l in mut_pos_1:
        if new_gens[names[0]][l] == 0: #If the value was 0, turn it into 1
            new_gens[names[0]][l] = 1
        else:
            new_gens[names[0]][l] = 0 #Otherwise turn it into 0
    mut_pos_2 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False)
    for l in mut_pos_2:
        if new_gens[names[1]][l] == 0: # Se era 0 vira 1
            new_gens[names[1]][l] = 1
        else:
            new_gens[names[1]][l] = 0
    mut_pos_3 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False)
    for l in mut_pos_3:
        if new_gens[names[2]][l] == 0: # Se era 0 vira 1
            new_gens[names[2]][l] = 1
        else:
            new_gens[names[2]][l] = 0
    mut_pos_4 = np.random.choice(int(np.minimum(size_1,size_2)), mut_num, replace=False)
    for l in mut_pos_4:
        if new_gens[names[3]][l] == 0: # Se era 0 vira 1
            new_gens[names[3]][l] = 1
        else:
            new_gens[names[3]][l] = 0
    return new_gens, new_state_space

In [51]:
start_time = time.time()
state_space = {} #espaço de estados conhecidos da espécie i
gens = {} #espaço de ações escolhidas pela espécie i
names = ['spec_%s' %(str(i)) for i in range (1,5)] #nomes das espécies
#criando as políticas das espécies originais
for i in range(4):
    gens[names[i]] = []
    state_space[names[i]] = []

#rodando as espécies:
while True:
    print('gens before game')
    print(gens)
    gens, state_space = teste(gens,state_space,names,start_time) #achar as relações entre estado e ações de cada espécie
    print('gens after game')
    print(gens)
    survivors = find_survivors(gens, names) #achar os sobreviventes
    print('survivors')
    print(survivors)
    gens, state_space = new_generation(gens, survivors, names, state_space) #nova geração
    print('new generation gens')
    print(gens)
    if time.time() - start_time > 60: #tempo máximo do loop
        break

gens before game
{'spec_1': [], 'spec_2': [], 'spec_3': [], 'spec_4': []}


KeyboardInterrupt: 